# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [14]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os

# Import API key
from api_keys import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

# Import the Configure gmaps, apply Google Key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [15]:
# Import csv file from the output_data folder under WeatherPy and display
csvpath = '../output_data/cities.csv'
cities_df = pd.read_csv(csvpath, index_col=0)
cities_df

,City,Latitude,Longitude,Max Temperature (°C),Humidity (%),Cloudiness (%),Wind Speed (MPH),Country,Date
0,meulaboh,4.1363,96.1285,26.86,80,100,1.28,ID,1627051550
1,esperance,-33.8667,121.9000,8.91,75,95,10.89,AU,1627051551
2,saskylakh,71.9167,114.0833,16.30,77,39,2.48,RU,1627051552
3,wanaka,-44.7000,169.1500,1.73,99,19,0.89,NZ,1627051554
4,vaini,-21.2000,-175.2000,24.09,94,75,2.06,TO,1627051555
...,...,...,...,...,...,...,...,...,...
604,xai-xai,-25.0519,33.6442,16.95,59,43,7.52,MZ,1627052379
605,apollonia,36.9716,24.7231,27.69,67,0,12.22,GR,1627052380
606,raub,3.7899,101.8570,24.98,84,94,0.99,MY,1627052381
607,baker city,44.7749,-117.8344,18.69,19,0,1.44,US,1627052383


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [16]:
# NOTE:  Configure gmaps was already added in the first cell

In [17]:
# Use Latitude and Longitude as the locations, and humidity as weights for the heat layer
locations = cities_df[["Latitude", "Longitude"]]
humidity = cities_df['Humidity (%)'].astype(float)

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
